In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
farms = pd.read_csv("../data-mining/farms.csv")
rubrics = pd.read_csv("../data-mining/rubrics.csv")
stocks = pd.read_csv("../data-mining/stocks.csv")

## Add departement column

In [19]:
def departement_of(code_postal: int) -> int:
    departement = int(str(code_postal)[:2])

    # dom_tom departement code is 3 digits-long instead of 2
    dom_tom: bool = departement == 97
    if dom_tom:
        departement = int(str(code_postal)[:3])

    return departement

farms["departement"] = farms.codePostal.apply(departement_of)

In [13]:
farms.head()

,adresse1,codePostal,codeInsee,commune,longitude,latitude,carriere,eolienne,industrie,prioriteNationale,...,etatActivite,regimeVigueur,siret,type,mise_en_demeure,name,codeNaf,adresse2,adresse3,departement
0,FERME DE FLASSY,89520,89331,SAINPUITS,3.257254,47.491203,False,False,False,False,...,En exploitation avec titre,Enregistrement,3.813267e+13,PORCS,False,GAEC DES BOURRYS,NaN,NaN,NaN,89
1,Pech Louis,81220,81266,ST PAUL CAP DE JOUX,1.988077,43.639220,False,False,False,False,...,En fin d'exploitation,Autorisation,3.284116e+13,PORCS,False,GAEC PORC D'ALGRES,NaN,NaN,NaN,81
2,LE HAMEL LELEGARD,50410,50393,PERCY-EN-NORMANDIE,-1.225119,48.913919,False,False,False,False,...,En exploitation avec titre,Enregistrement,8.796621e+13,BOVINS,False,GAEC FERME H ET L,1.0,NaN,NaN,50
3,NaN,50410,50393,PERCY EN NORMANDIE,-1.187543,48.950369,False,False,False,False,...,En exploitation avec titre,Autorisation,4.233382e+13,BOVINS,False,BINOUDIERE (GAEC DE LA),NaN,La Binoudière,NaN,50
4,NaN,50410,50393,PERCY EN NORMANDIE,-1.253302,48.918864,False,False,False,False,...,En exploitation avec titre,Enregistrement,4.083231e+13,BOVINS,False,BOCAGE (GAEC DU),NaN,LE HAMEL AUX LOUVEAUX,NaN,50


In [3]:
stocks.head()

,raison_sociale,rubric_id,quantite,unite
0,GAEC DES BOURRYS,2102,1069.0,Animaux-Eq
1,GAEC PORC D'ALGRES,3660,860.0,NaN
2,GAEC PORC D'ALGRES,2102,860.0,NaN
3,GAEC FERME H ET L,2101,105.0,NaN
4,GAEC FERME H ET L,2101,150.0,NaN


In [5]:
farms.merge(stocks, left_on="name", right_on="raison_sociale")

,adresse1,codePostal,codeInsee,commune,longitude,latitude,carriere,eolienne,industrie,prioriteNationale,...,type,mise_en_demeure,name,codeNaf,adresse2,adresse3,raison_sociale,rubric_id,quantite,unite
0,FERME DE FLASSY,89520,89331,SAINPUITS,3.257254,47.491203,False,False,False,False,...,PORCS,False,GAEC DES BOURRYS,NaN,NaN,NaN,GAEC DES BOURRYS,2102,1069.0,Animaux-Eq
1,Pech Louis,81220,81266,ST PAUL CAP DE JOUX,1.988077,43.639220,False,False,False,False,...,PORCS,False,GAEC PORC D'ALGRES,NaN,NaN,NaN,GAEC PORC D'ALGRES,3660,860.0,NaN
2,Pech Louis,81220,81266,ST PAUL CAP DE JOUX,1.988077,43.639220,False,False,False,False,...,PORCS,False,GAEC PORC D'ALGRES,NaN,NaN,NaN,GAEC PORC D'ALGRES,2102,860.0,NaN
3,LE HAMEL LELEGARD,50410,50393,PERCY-EN-NORMANDIE,-1.225119,48.913919,False,False,False,False,...,BOVINS,False,GAEC FERME H ET L,1.0,NaN,NaN,GAEC FERME H ET L,2101,105.0,NaN
4,LE HAMEL LELEGARD,50410,50393,PERCY-EN-NORMANDIE,-1.225119,48.913919,False,False,False,False,...,BOVINS,False,GAEC FERME H ET L,1.0,NaN,NaN,GAEC FERME H ET L,2101,150.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23682,La Clinerie,45250,45245,OUZOUER SUR TREZEE,2.803794,47.655749,False,False,False,False,...,VOLAILLES,False,DE LA CLINERIE (GAEC) LEFEVRE Christian,NaN,NaN,NaN,DE LA CLINERIE (GAEC) LEFEVRE Christian,2111,25500.0,Animaux-Eq
23683,La Clinerie,45250,45245,OUZOUER SUR TREZEE,2.803794,47.655749,False,False,False,False,...,VOLAILLES,False,DE LA CLINERIE (GAEC) LEFEVRE Christian,NaN,NaN,NaN,DE LA CLINERIE (GAEC) LEFEVRE Christian,2111,25500.0,NaN
23684,LES JOURNETS,41160,41216,ST JEAN FROIDMENTEL,1.219027,47.962486,False,False,False,False,...,PORCS,False,DES JOURNETS (SCEA),NaN,NaN,NaN,DES JOURNETS (SCEA),2102,1857.0,Animaux-Eq
23685,LES MUSSETS,41160,41028,BUSLOUP,1.132000,47.908176,False,False,False,False,...,VOLAILLES,False,EARL DES MUSSETS,NaN,NaN,NaN,EARL DES MUSSETS,2111,40000.0,u
